In [17]:
import requests, json
from urllib.parse import urlencode, quote
from Crypto.Util.Padding import pad, unpad
GET = lambda url: json.loads(requests.get(url).text)

def Get_cookie():
    return GET("https://aes.cryptohack.org/flipping_cookie/get_cookie/").get("cookie")
def GetFlag(cookie, iv):
    try:
        res = GET(f"https://aes.cryptohack.org/flipping_cookie/check_admin/{quote(cookie)}/{quote(iv)}/")
        return res
    except:
        return requests.get(f"https://aes.cryptohack.org/flipping_cookie/check_admin/{quote(cookie)}/{quote(iv)}/").text

In [2]:
GET("https://aes.cryptohack.org/flipping_cookie/get_cookie/")

{'cookie': 'de3255145442636c8a1e80b5f7bdd7964113b5acbb671802c6cdd08ddb8eb33e107da57638bc6a1645d91605f006a0a3'}

In [14]:
def hexToArray(hexstring):
    return [int(hexstring[i:i+2], 16) for i in range(0, len(hexstring), 2)]
    # return [int(_) for _ in (bytes.fromhex(hexstring) if isinstance(hexstring, str) else hexstring)]

def bitwise_xor(str1, str2):
    if isinstance(str1, str):
        str1 = hexToArray(str1)
    if isinstance(str2, str):
        str2 = hexToArray(str2)
    return [a^b for a, b in zip(str1, str2)]

def bitwise_xors(*strs):
    if len(strs) < 2: return None
    tmp = strs[0]
    for _str in strs[1:]:
        tmp = bitwise_xor(_str, tmp)
    return tmp

def intList_to_str(intList):
    return "".join((chr(_) for _ in intList))
string = intList_to_str

def intList_to_hex(intList):
    return "".join(hex(_)[2:] for _ in intList)

def slice_half(iter):
    return iter[:len(iter)//2], iter[len(iter)//2:]

In [4]:
COOKIE = Get_cookie()
iv, cookie = COOKIE[:32], COOKIE[32:]

In [5]:
iv, cookie

('21d1473db8d9e73520cabaa3ed8ec07f',
 '56ca12fc169806b510452f31709c5d218bb4f44dda2e43961de265125dd97873')

In [21]:
i = 3
a = str(i).zfill(2)
malicious = pad((f"admin=True;").encode(), 16)
mal_iv_01 = bitwise_xor(malicious, iv)
malicious_iv = intList_to_hex(bitwise_xor(pad((f"admin=False;").encode(), 16), mal_iv_01))
print(a, cookie, malicious_iv)
GetFlag(cookie, malicious_iv)

03 56ca12fc169806b510452f31709c5d218bb4f44dda2e43961de265125dd97873 21d1473db8d9f52639dce49dec8fc17e


{'flag': 'crypto{4u7h3n71c4710n_15_3553n714l}'}

In [92]:
GetFlag(cookie, iv)

'NONE'

In [68]:
plaintext = hexToArray(pad(b"admin=False;", 16)), hexToArray(pad(b'', 16))
ciphertext = [hexToArray(_) for _ in slice_half(cookie)]
plaintext, ciphertext, iv

(([97, 100, 109, 105, 110, 61, 70, 97, 108, 115, 101, 59, 4, 4, 4, 4],
  [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]),
 [[163, 41, 4, 88, 8, 76, 120, 119, 28, 142, 126, 42, 128, 84, 203, 200],
  [204, 185, 235, 130, 110, 236, 187, 55, 6, 205, 160, 48, 151, 195, 41, 109]],
 '36463ea67259b6227b1252fb68d5a133')

In [90]:
malicious = pad(b"admin=True;", 16)

cookie, intList_to_hex(bitwise_xors(ciphertext[0], malicious, iv))

('a3290458084c78771c8e7e2a8054cbc8ccb9eb826eecbb3706cda03097c3296d',
 'f4b579714289a2712f917d4ed846ffe')

In [91]:
len("f4b579714289a2712f917d4ed846ffe")

31

In [80]:
GetFlag(cookie, intList_to_hex(bitwise_xors(plaintext[0], malicious,  iv)))

'NONE'

In [36]:
bitwise_xor(ciphertext, iv)

TypeError: unsupported operand type(s) for ^: 'bytes' and 'int'

In [29]:
bitwise_xor(plaintext[1], cookie[0])

[163, 5, 131, 75, 13, 132, 232, 235, 62, 123, 120, 159, 72, 41, 244, 130]

In [10]:
cookies = slice_half(hexToArray(cookie))
iv = hexToArray(iv)

In [11]:
bitwise_xor(cookies[0], iv)

[248, 22, 251, 45, 79, 245, 68, 181, 124, 182, 114, 195, 130, 102, 1, 227]

In [27]:
a=b"admin=True;;expiry=02"

print(pad(a, 16), pad(b'', 16))
bitwise_xor(pad(a, 16), iv)

b'admin=True;;expiry=02\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b' b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'


[47, 225, 123, 151, 192, 127, 4, 142, 93, 149, 55, 136, 151, 56, 159, 98]